In [1]:
import sys
sys.path.append("../../libs")
from models import *

In [2]:
import logging

from autogen_core.logging import LLMCallEvent


class LLMUsageTracker(logging.Handler):
    def __init__(self) -> None:
        """Logging handler that tracks the number of tokens used in the prompt and completion."""
        super().__init__()
        self._prompt_tokens = 0
        self._completion_tokens = 0

    @property
    def tokens(self) -> int:
        return self._prompt_tokens + self._completion_tokens

    @property
    def prompt_tokens(self) -> int:
        return self._prompt_tokens

    @property
    def completion_tokens(self) -> int:
        return self._completion_tokens

    def reset(self) -> None:
        self._prompt_tokens = 0
        self._completion_tokens = 0

    def emit(self, record: logging.LogRecord) -> None:
        """Emit the log record. To be used by the logging module."""
        try:
            # Use the StructuredMessage if the message is an instance of it
            if isinstance(record.msg, LLMCallEvent):
                event = record.msg
                self._prompt_tokens += event.prompt_tokens
                self._completion_tokens += event.completion_tokens
        except Exception:
            self.handleError(record)

In [5]:
from autogen_core import EVENT_LOGGER_NAME

# Set up the logging configuration to use the custom handler
logger = logging.getLogger(EVENT_LOGGER_NAME)
logger.setLevel(logging.INFO)
llm_usage = LLMUsageTracker()
logger.handlers = [llm_usage]

# client.create(...)
res = await chat("你好，请讲个笑话", litellm_llama32_1b())
print(res.content)

print(llm_usage.prompt_tokens)
print(llm_usage.completion_tokens)

有一个老人走入一家餐厅，发现餐厅里有一位穿着 formal 的女士。老人的脑子里突然 flashed 一下感慨：“这位老人可能是我的长辈啊。”但又不想暴露自己的真实身份，所以他说：“你是有名的医生吗？”女士 reply “我不是医生，偶尔做一些研究工作”，老人听了，很是感动。
36
101
